In [1]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import lightgbm as lgb
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [2]:
train_data = pd.read_csv("duom/train_data.csv")
test_data = pd.read_csv("duom/test_data.csv")

########################################
Išrinamos kategorinės reikšmės, kurių nėra testiniame duomenų rinkinyje
########################################

In [3]:
for column in train_data:
    if train_data[column].dtype == "object":
        print(column)
        print(train_data[column].unique())
        print(len(train_data[column].unique()))
        print(set(train_data[column].unique()).difference(set(test_data[column].unique())))
        print(set(test_data[column].unique()).difference(set(train_data[column].unique())))
        print("")

term
['short' 'long']
2
set()
set()

credit_score
['very_good' 'good' nan 'fair']
4
set()
set()

loan_purpose
['other' 'debt_consolidation' 'home_improvements' 'take_a_trip'
 'medical_bills' 'educational_expenses' 'major_purchase' 'small_business'
 'business_loan' 'buy_a_car' 'moving' 'buy_house' 'vacation' 'wedding'
 'renewable_energy' 'UFVCU1VC']
16
{'renewable_energy', 'UFVCU1VC'}
set()

home_ownership
['rent' 'mortgage' 'own' nan]
4
{nan}
set()



In [4]:
train_data.drop(train_data[train_data['loan_purpose'] == 'renewable_energy'].index, inplace=True)
train_data.drop(train_data[train_data['loan_purpose'] == 'UFVCU1VC'].index, inplace=True)

########################################
Trūkstamų reikšmių tikrinimas
########################################

In [5]:
train_data.isna().sum()

id                                    0
y                                     0
amount_current_loan                   0
term                                  0
credit_score                    2096226
loan_purpose                          0
yearly_income                   1687689
home_ownership                        1
bankruptcies                      16195
years_current_job                371020
monthly_debt                          1
years_credit_history                  1
months_since_last_delinquent    4665726
open_accounts                         1
credit_problems                       1
credit_balance                        1
max_open_credit                     175
dtype: int64

########################################
Ištrinamos arba užpildomos trūkstamos reikšmės
########################################

In [6]:
train_data.drop(train_data[(train_data['credit_score'].isna()) 
     & (train_data['months_since_last_delinquent'].isna()) 
     & (train_data['yearly_income'].isna())].index, inplace=True)

train_data.drop(train_data[train_data["max_open_credit"].isnull()].index, inplace = True) 
train_data.drop(train_data[train_data["bankruptcies"].isnull()].index, inplace = True) 
train_data.drop(train_data[train_data["years_current_job"].isnull()].index, inplace = True) 
#train_data.drop(train_data[train_data["credit_score"].isnull()].index, inplace = True) 

train_data["months_since_last_delinquent"] = train_data["months_since_last_delinquent"].fillna(value = 200)
test_data["months_since_last_delinquent"] = test_data["months_since_last_delinquent"].fillna(value = 200)

train_data["credit_score"] = train_data["credit_score"].fillna(value = "unknown")
test_data["credit_score"] = test_data["credit_score"].fillna(value = "unknown")

train_data.drop(train_data[train_data["yearly_income"].isnull()].index, inplace = True) 

In [7]:
train_data.isna().sum()

id                              0
y                               0
amount_current_loan             0
term                            0
credit_score                    0
loan_purpose                    0
yearly_income                   0
home_ownership                  0
bankruptcies                    0
years_current_job               0
monthly_debt                    0
years_credit_history            0
months_since_last_delinquent    0
open_accounts                   0
credit_problems                 0
credit_balance                  0
max_open_credit                 0
dtype: int64

In [8]:
train_data

,id,y,amount_current_loan,term,credit_score,loan_purpose,yearly_income,home_ownership,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
0,1,0,386342,short,very_good,other,1371971.0,rent,0.0,4.0,5281.62,17.0,25.0,9.0,0.0,59356.0,388828.0
1,2,0,429000,short,good,debt_consolidation,823042.0,mortgage,0.0,10.0,13854.61,19.0,10.0,15.0,0.0,344584.0,811778.0
4,5,0,654126,short,good,debt_consolidation,1242847.0,rent,0.0,0.0,27490.15,28.0,68.0,8.0,1.0,265259.0,456214.0
5,6,0,133034,short,good,debt_consolidation,651035.0,rent,0.0,10.0,6473.11,30.0,200.0,12.0,0.0,102771.0,387662.0
6,7,0,317570,short,good,debt_consolidation,1937601.0,mortgage,0.0,10.0,34154.97,17.0,200.0,12.0,0.0,171684.0,240438.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8774102,8774103,1,220814,short,good,debt_consolidation,1430282.0,mortgage,0.0,2.0,27378.62,14.0,200.0,16.0,0.0,393984.0,558822.0
8774103,8774104,0,197978,short,very_good,debt_consolidation,2090836.0,mortgage,1.0,6.0,14189.01,21.0,40.0,15.0,1.0,159372.0,510400.0
8774106,8774107,1,176308,short,unknown,buy_a_car,1807888.0,own,1.0,4.0,23347.58,21.0,200.0,15.0,1.0,65512.0,352352.0
8774110,8774111,0,142010,short,good,debt_consolidation,766688.0,mortgage,0.0,10.0,15732.38,18.0,70.0,6.0,0.0,370899.0,572748.0


In [9]:
train_data.isna().sum()

id                              0
y                               0
amount_current_loan             0
term                            0
credit_score                    0
loan_purpose                    0
yearly_income                   0
home_ownership                  0
bankruptcies                    0
years_current_job               0
monthly_debt                    0
years_credit_history            0
months_since_last_delinquent    0
open_accounts                   0
credit_problems                 0
credit_balance                  0
max_open_credit                 0
dtype: int64

In [10]:
train_data.describe().applymap(lambda x: f"{x:0.3f}")

,id,y,amount_current_loan,yearly_income,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
count,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000
mean,4404564.259,0.233,14348760.093,1398613.710,0.113,5.908,18749.953,18.016,122.191,11.206,0.170,301592.844,772009.987
std,2525270.420,0.423,34666851.236,1148568.160,0.346,3.630,12251.730,6.831,83.800,5.060,0.490,384432.766,7562691.566
min,1.000,0.000,10802.000,76627.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000
25%,2223847.000,0.000,192522.000,861954.000,0.000,3.000,10457.790,13.000,34.000,8.000,0.000,114456.000,278014.000
50%,4406973.000,0.000,326612.000,1201446.000,0.000,6.000,16477.940,17.000,200.000,10.000,0.000,213427.000,475398.000
75%,6590606.500,0.000,551320.000,1677168.000,0.000,10.000,24302.330,22.000,200.000,14.000,0.000,375345.000,797104.000
max,8774112.000,1.000,99999999.000,165557393.000,7.000,10.000,435843.280,70.000,200.000,76.000,15.000,32878968.000,1539737892.000


In [11]:
train_data.describe().applymap(lambda x: f"{x:0.3f}")

,id,y,amount_current_loan,yearly_income,bankruptcies,years_current_job,monthly_debt,years_credit_history,months_since_last_delinquent,open_accounts,credit_problems,credit_balance,max_open_credit
count,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000,6773899.000
mean,4404564.259,0.233,14348760.093,1398613.710,0.113,5.908,18749.953,18.016,122.191,11.206,0.170,301592.844,772009.987
std,2525270.420,0.423,34666851.236,1148568.160,0.346,3.630,12251.730,6.831,83.800,5.060,0.490,384432.766,7562691.566
min,1.000,0.000,10802.000,76627.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000
25%,2223847.000,0.000,192522.000,861954.000,0.000,3.000,10457.790,13.000,34.000,8.000,0.000,114456.000,278014.000
50%,4406973.000,0.000,326612.000,1201446.000,0.000,6.000,16477.940,17.000,200.000,10.000,0.000,213427.000,475398.000
75%,6590606.500,0.000,551320.000,1677168.000,0.000,10.000,24302.330,22.000,200.000,14.000,0.000,375345.000,797104.000
max,8774112.000,1.000,99999999.000,165557393.000,7.000,10.000,435843.280,70.000,200.000,76.000,15.000,32878968.000,1539737892.000


In [13]:
train_data.to_csv("duom/train_data_1.csv", sep=',', encoding='utf-8', index=False)
test_data.to_csv("duom/test_data_1.csv", sep=',', encoding='utf-8', index=False)